In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import importlib
from IPython.display import HTML

from src.functions import *
from src.config import *
from src.allocating import*
from src.particles import *
from src.dimensionless_scaling_test import *
from src.calculating_observables import *
import src.functions
#importlib.reload(src.functions)

#####initialize simulation######
fig = plt.figure()

#allocate memory
pos=allocating_variables()                                                     
radii = size_of_particles(N_particles, 1, 0.1)
colours_particles = np.zeros(shape=(N_particles,N_steps),dtype='object')
colours_orientation = np.zeros(shape=(N_particles, N_steps),dtype='object')
orientation = np.zeros((N_particles, N_steps))

# initialize variables
pos[:,:,0]=cubic_latice(N_particles_x,N_particles_y)
colours_particles[:,:] = '-g' 
colours_orientation[:,:] = '-b'

scaling_force,scaling_torque,lambda_a,lambda_s,lambda_n,lambda_F_in,lambda_T_in=func_dimensionless_scaling(radii)



orientation[:,0] = orientation_at_0(N_particles) # orientation in radians!
print(orientation[:,0])
###perform simulation
for time_step in range(1,N_steps):
    print("step is",time_step)
    pos[:,:,time_step], orientation[:,time_step], colours_particles[:,time_step],colours_orientation[:,time_step],displacement = update_position_and_orientation(
        pos[:,:,time_step-1], 
        orientation[:,time_step-1],
        radii,
        colours_particles[:,time_step-1], 
        colours_orientation[:,time_step-1],time_size,scaling_force,scaling_torque,lambda_a,lambda_s,lambda_n,lambda_F_in,lambda_T_in)

# dataprocessing
orientation = np.radians(orientation)
radius = np.reshape(radii, (N_particles,1))
orientation_x = pos[:,0,:]+radius*np.cos(orientation)
orientation_y = pos[:,1,:]+radius*-np.sin(orientation)

print('data done')

# calculating observables
ACV=auto_correlation_velocity(pos,timestep=1)#what is the timestepsize?
MSD=mean_square_displacement(pos)
# diffusion stil needs testing
center_mass_in_time=center_of_mass(pos,radii,time_step)
Diffusion=func_diffusion(center_mass_in_time,time_size)

####create plot ####
lns = []
trans = plt.axes().transAxes
w = np.linspace(1,10,N_steps)
for steps in range (N_steps): # loop through timesteps
    x,y = particle(pos[:,0,steps], pos[:,1,steps], radii[:], N_particles)
    lns_timestep = []
    if (steps % 300) == 0:
        for i in range(N_particles): #loop through particles (want to get rid of 
                                     # this, how can we do this?)
            ln1, = plt.plot(x[:,i], y[:,i], colours_particles[i,steps], lw=2)

            ln2, = plt.plot([pos[i,0,steps],orientation_x[i,steps]],[pos[i,1,steps],orientation_y[i,steps]], colours_orientation[i,steps],lw=2)

            lns_timestep.append(ln1)
            lns_timestep.append(ln2)
            #lns_timestep.append(ln3)
        lns.append(lns_timestep)
        print("step animation is:",steps," out of ",N_steps)

plt.figure()

plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.axes().set_aspect('equal')

ani = animation.ArtistAnimation(fig, lns, interval=100/time_size,blit=True)
#ani.save("test.html")
HTML(ani.to_html5_video())

#plt.show()



[ 25.83875796  16.34581594 -33.92902299  40.0676436    1.57023194
 -40.52416712 -44.89576227 -16.45409251 -39.07182682  25.33840686
  13.61645351 -25.9260647  -15.98336589  14.24106154 -30.07052502
 -24.8208917   31.71474723 -22.77798199  39.20466339  22.84401826
  -8.29766406   1.90852623 -40.18500063 -35.4191392  -35.02468945
   0.67180347 -36.02342987  17.85238702 -10.26361311  30.06411676
  14.45735788 -38.11500248  15.86706205  -9.36178921  18.71860278
 -37.97237101 -35.72462162  -4.63973626  24.45089633   5.22484601
 -23.08815921 -27.19384769  28.76519962 -43.59767669  11.86049163
 -15.55595616  22.71843031  -5.61787467  26.10699212 -13.16229159
  29.52540882 -39.3492885  -39.65354887  16.08020685  41.35537873
 -41.01708493  33.82589364 -19.01873728  20.7166674    8.62008408
   3.71628308 -17.43108259  35.50280349  35.15625337 -31.14884611
 -15.85597856  43.82880356  31.60955641  31.35221351 -15.77044871
 -18.85200884  -9.59698813  31.21992254  19.30259551  -2.71039265
  27.81247

/Users/peter-janderks/Desktop/computational_physics/project_3/master/src/functions.py:191: RuntimeWarning: invalid value encountered in true_divide
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2-R2**2)/(2*d*R1))+R2**2*np.arccos((d**2-R1**2+R2**2)/(2*d*R2))-(1/2)*np.sqrt((-d+R1+R2)*(d+R1-R2)*(d-R1+R2)*(d+R1+R2)),0)
/Users/peter-janderks/Desktop/computational_physics/project_3/master/src/functions.py:191: RuntimeWarning: invalid value encountered in arccos
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2-R2**2)/(2*d*R1))+R2**2*np.arccos((d**2-R1**2+R2**2)/(2*d*R2))-(1/2)*np.sqrt((-d+R1+R2)*(d+R1-R2)*(d-R1+R2)*(d+R1+R2)),0)
/Users/peter-janderks/Desktop/computational_physics/project_3/master/src/functions.py:191: RuntimeWarning: invalid value encountered in sqrt
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2

step is 7
step is 8
step is 9
step is 10
step is 11
step is 12
step is 13
step is 14
step is 15
step is 16
step is 17
step is 18
step is 19
step is 20
step is 21
step is 22
step is 23
step is 24
step is 25
step is 26
step is 27
step is 28
step is 29
step is 30
step is 31
step is 32
step is 33
step is 34
step is 35
step is 36
step is 37
step is 38
step is 39
step is 40
step is 41
step is 42
step is 43
step is 44
step is 45
step is 46
step is 47
step is 48
step is 49
step is 50
step is 51
step is 52
step is 53
step is 54
step is 55
step is 56
step is 57
step is 58
step is 59
step is 60
step is 61
step is 62
step is 63
step is 64
step is 65
step is 66
step is 67
step is 68
step is 69
step is 70
step is 71
step is 72
step is 73
step is 74
step is 75
step is 76
step is 77
step is 78
step is 79
step is 80
step is 81
step is 82
step is 83
step is 84
step is 85
step is 86
step is 87
step is 88
step is 89
step is 90
step is 91
step is 92
step is 93
step is 94
step is 95
step is 96
step is 97
st

step is 703
step is 704
step is 705
step is 706
step is 707
step is 708
step is 709
step is 710
step is 711
step is 712
step is 713
step is 714
step is 715
step is 716
step is 717
step is 718
step is 719
step is 720
step is 721
step is 722
step is 723
step is 724
step is 725
step is 726
step is 727
step is 728
step is 729
step is 730
step is 731
step is 732
step is 733
step is 734
step is 735
step is 736
step is 737
step is 738
step is 739
step is 740
step is 741
step is 742
step is 743
step is 744
step is 745
step is 746
step is 747
step is 748
step is 749
step is 750
step is 751
step is 752
step is 753
step is 754
step is 755
step is 756
step is 757
step is 758
step is 759
step is 760
step is 761
step is 762
step is 763
step is 764
step is 765
step is 766
step is 767
step is 768
step is 769
step is 770
step is 771
step is 772
step is 773
step is 774
step is 775
step is 776
step is 777
step is 778
step is 779
step is 780
step is 781
step is 782
step is 783
step is 784
step is 785
step

step is 1358
step is 1359
step is 1360
step is 1361
step is 1362
step is 1363
step is 1364
step is 1365
step is 1366
step is 1367
step is 1368
step is 1369
step is 1370
step is 1371
step is 1372
step is 1373
step is 1374
step is 1375
step is 1376
step is 1377
step is 1378
step is 1379
step is 1380
step is 1381
step is 1382
step is 1383
step is 1384
step is 1385
step is 1386
step is 1387
step is 1388
step is 1389
step is 1390
step is 1391
step is 1392
step is 1393
step is 1394
step is 1395
step is 1396
step is 1397
step is 1398
step is 1399
step is 1400
step is 1401
step is 1402
step is 1403
step is 1404
step is 1405
step is 1406
step is 1407
step is 1408
step is 1409
step is 1410
step is 1411
step is 1412
step is 1413
step is 1414
step is 1415
step is 1416
step is 1417
step is 1418
step is 1419
step is 1420
step is 1421
step is 1422
step is 1423
step is 1424
step is 1425
step is 1426
step is 1427
step is 1428
step is 1429
step is 1430
step is 1431
step is 1432
step is 1433
step is 1434

...

step is 8329
step is 8330
step is 8331
step is 8332
step is 8333
step is 8334
step is 8335
step is 8336
step is 8337
step is 8338
step is 8339
step is 8340
step is 8341
step is 8342
step is 8343
step is 8344
step is 8345
step is 8346
step is 8347
step is 8348
step is 8349
step is 8350
step is 8351
step is 8352
step is 8353
step is 8354
step is 8355
step is 8356
step is 8357
step is 8358
step is 8359
step is 8360
step is 8361
step is 8362
step is 8363
step is 8364
step is 8365
step is 8366
step is 8367
step is 8368
step is 8369
step is 8370
step is 8371
step is 8372
step is 8373
step is 8374
step is 8375
step is 8376
step is 8377
step is 8378
step is 8379
step is 8380
step is 8381
step is 8382
step is 8383
step is 8384
step is 8385
step is 8386
step is 8387
step is 8388
step is 8389
step is 8390
step is 8391
step is 8392
step is 8393
step is 8394
step is 8395
step is 8396
step is 8397
step is 8398
step is 8399
step is 8400
step is 8401
step is 8402
step is 8403
step is 8404
step is 8405

step is 8963
step is 8964
step is 8965
step is 8966
step is 8967
step is 8968
step is 8969
step is 8970
step is 8971
step is 8972
step is 8973
step is 8974
step is 8975
step is 8976
step is 8977
step is 8978
step is 8979
step is 8980
step is 8981
step is 8982
step is 8983
step is 8984
step is 8985
step is 8986
step is 8987
step is 8988
step is 8989
step is 8990
step is 8991
step is 8992
step is 8993
step is 8994
step is 8995
step is 8996
step is 8997
step is 8998
step is 8999
step is 9000
step is 9001
step is 9002
step is 9003
step is 9004
step is 9005
step is 9006
step is 9007
step is 9008
step is 9009
step is 9010
step is 9011
step is 9012
step is 9013
step is 9014
step is 9015
step is 9016
step is 9017
step is 9018
step is 9019
step is 9020
step is 9021
step is 9022
step is 9023
step is 9024
step is 9025
step is 9026
step is 9027
step is 9028
step is 9029
step is 9030
step is 9031
step is 9032
step is 9033
step is 9034
step is 9035
step is 9036
step is 9037
step is 9038
step is 9039

step is 9596
step is 9597
step is 9598
step is 9599
step is 9600
step is 9601
step is 9602
step is 9603
step is 9604
step is 9605
step is 9606
step is 9607
step is 9608
step is 9609
step is 9610
step is 9611
step is 9612
step is 9613
step is 9614
step is 9615
step is 9616
step is 9617
step is 9618
step is 9619
step is 9620
step is 9621
step is 9622
step is 9623
step is 9624
step is 9625
step is 9626
step is 9627
step is 9628
step is 9629
step is 9630
step is 9631
step is 9632
step is 9633
step is 9634
step is 9635
step is 9636
step is 9637
step is 9638
step is 9639
step is 9640
step is 9641
step is 9642
step is 9643
step is 9644
step is 9645
step is 9646
step is 9647
step is 9648
step is 9649
step is 9650
step is 9651
step is 9652
step is 9653
step is 9654
step is 9655
step is 9656
step is 9657
step is 9658
step is 9659
step is 9660
step is 9661
step is 9662
step is 9663
step is 9664
step is 9665
step is 9666
step is 9667
step is 9668
step is 9669
step is 9670
step is 9671
step is 9672

/Users/peter-janderks/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyError: 'ffmpeg'

In [7]:
deviation_0 = np.random.uniform(-0.785*180/np.pi, 0.785*180/np.pi, N_particles)
#deviation_0[deviation_0>0] = 360 - deviation_0[deviation_0>0]
print(abs(deviation_0))

[ 37.61247798   2.44430197  20.99159239   1.75793696  27.35468163
  37.19104393  24.96963501  27.03119709  37.31550799  14.93264061
  14.96432889  33.4001864   44.05470547  17.6505087   14.13263705
   0.19035793   5.515392     1.86946732  33.26376675  17.69033482
  40.98842394  30.32030185  16.15778703  39.74963013  10.68417687]


In [5]:
        u = np.array([orientation_x, orientation_y])
        v = np.array([init_orien_x[steps,i]+pos[i,0,steps], init_orien_y[steps,i]+pos[i,1,steps]])
        len_u = np.linalg.norm(u)
        len_v = np.linalg.norm(v)
        cosx = np.dot(u,v)/(len_u*len_v)
        delta_degrees = np.arccos(cosx)*180/np.pi
        if steps == 2:
            print(delta_degrees, i, 'delta_theta' )
        

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [2]:
print(8**4)

4096
